# Imports necesaris 


In [1]:
#pip install opencv-python
#pip install tensorflow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import (Input, Add, Dense, Activation,
ZeroPadding2D, BatchNormalization, Flatten, Conv2D, 
AveragePooling2D, MaxPooling2D, Dropout)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import (ReduceLROnPlateau,
EarlyStopping, ModelCheckpoint, LearningRateScheduler)
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objects as go

### Mini-reto 5

In [3]:
df = pd.read_csv('DataSet/sales_data_sample.csv', sep=',', decimal='.', encoding = "ISO-8859-1")
df.groupby('COUNTRY').size()
#df["ORDERDATE"] = pd.to_datetime(df["ORDERDATE"])
df = df.drop(["PHONE", "ADDRESSLINE1", "ADDRESSLINE2", "POSTALCODE", "CONTACTLASTNAME", "CONTACTFIRSTNAME"], axis=1)
df

#MINIRETO 4.1
print(len(df['STATE'].value_counts().index))
#MINIRETO 4.2

#Ara per ara no creiem necessari balancejar cap camp per l'ús que els hi volem donar. No obstant No ens tanquem les portes a haver de fer-ho mes endavant si s'escau.
#Camps que creiem que poden necessitar balanceix: COUNTRY, ORDERLINENUMBER, ORDERNUMBER

#MINIRETO 4.3 i 4.4 Función para agregar variables ficticias para reemplazar variables categóricas
def modificarVariables(variables):
    for variable in variables:
        dicc = dict(zip(df.groupby(variable).groups, range(0, len(df.groupby(variable).groups))))
        df.replace({variable : dicc}, inplace=True)
        
modificarVariables(["STATUS", "PRODUCTLINE", "DEALSIZE", "COUNTRY"])

#MINIRETO 5
orderLine_dummy = pd.get_dummies(df["ORDERLINENUMBER"], prefix= "ORDERLINENUMBER")
df = pd.concat([df, orderLine_dummy], axis = 1)
df

16


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ORDERLINENUMBER_9,ORDERLINENUMBER_10,ORDERLINENUMBER_11,ORDERLINENUMBER_12,ORDERLINENUMBER_13,ORDERLINENUMBER_14,ORDERLINENUMBER_15,ORDERLINENUMBER_16,ORDERLINENUMBER_17,ORDERLINENUMBER_18
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,5,1,2,2003,...,0,0,0,0,0,0,0,0,0,0
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,5,2,5,2003,...,0,0,0,0,0,0,0,0,0,0
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,5,3,7,2003,...,0,0,0,0,0,0,0,0,0,0
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,5,3,8,2003,...,0,0,0,0,0,0,0,0,0,0
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,5,4,10,2003,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,5,4,12,2004,...,0,0,0,0,0,0,1,0,0,0
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,5,1,1,2005,...,0,0,0,0,0,0,0,0,0,0
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,4,1,3,2005,...,0,0,0,0,0,0,0,0,0,0
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,5,1,3,2005,...,0,0,0,0,0,0,0,0,0,0
